# Narrative Generator

From the events Wyscout dataset https://figshare.com/collections/Soccer_match_event_dataset/4415000/5, we can write a simple narrative generator to see what's happening during the game in a more natural way. It also helps understand the meaning of the event data, such as eventName, tags etc, especially if you can compare the narrative with the match video.

In [1]:
# load some packages
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import os

from utils import wyscout # package to deal with wyscout data

In [2]:
# load the dataset from https://figshare.com/collections/Soccer_match_event_dataset/4415000/5
# wyscout.download_data()

# use the Premier League as an example
dirname = './data/Wyscout/events'
fn = os.path.join(dirname, 'events_England.json')
season = '2017-2018'
league = 'Premier League'

# read in event data using pandas --> DataFrame
df_events = pd.read_json(fn)
df_events.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175


In [3]:
# matchId = np.random.choice(df_events['matchId'].unique()) # randomly extract a match
matchId = 2499754 # Man City - Man City, 5:0
match_events = df_events[df_events.matchId==matchId]

# get some match information
matchfile = './data/Wyscout/matches/matches_England.json'
df_match = pd.read_json(matchfile)
df_match[df_match['wyId']==matchId]

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId
349,Played,4405654,4,"{'1612': {'scoreET': 0, 'coachId': 14791, 'sid...",181150,2017-09-09 11:30:00,1625,Etihad Stadium,2499754,"Manchester City - Liverpool, 5 - 0","September 9, 2017 at 1:30:00 PM GMT+2","[{'refereeId': 381851, 'role': 'referee'}, {'r...",Regular,364


In [4]:
# add extra informative columns
# mapping teamId, playerId, tags to labels, and build time column with format (half, minute, sec)
match_events = wyscout.add_event_cols(match_events, ['team', 'player', 'taglist', 'time'])
match_events.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,team,player,taglist,time
56750,8,Simple pass,[{'id': 1801}],116,"[{'y': 48, 'x': 50}, {'y': 55, 'x': 27}]",2499754,Pass,1612,1H,0.841815,85,185702706,Liverpool,G. Wijnaldum,[Accurate],"(1, 0, 1)"
56751,8,Simple pass,[{'id': 1801}],14870,"[{'y': 55, 'x': 27}, {'y': 79, 'x': 41}]",2499754,Pass,1612,1H,2.864206,85,185702707,Liverpool,J. Matip,[Accurate],"(1, 0, 3)"
56752,8,Simple pass,[{'id': 1801}],346101,"[{'y': 79, 'x': 41}, {'y': 67, 'x': 70}]",2499754,Pass,1612,1H,6.850935,85,185702711,Liverpool,T. Alexander-Arnold,[Accurate],"(1, 0, 7)"
56753,8,Simple pass,[{'id': 1801}],120353,"[{'y': 67, 'x': 70}, {'y': 87, 'x': 55}]",2499754,Pass,1612,1H,8.040190,85,185757993,Liverpool,Mohamed Salah,[Accurate],"(1, 0, 8)"
56754,8,Simple pass,[{'id': 1801}],116,"[{'y': 87, 'x': 55}, {'y': 95, 'x': 66}]",2499754,Pass,1612,1H,9.988902,85,185702722,Liverpool,G. Wijnaldum,[Accurate],"(1, 0, 10)"


In [5]:
# filter the time window of the events, otherwise it will be about 1700 events
filtered_events = wyscout.filter_events_by_time(match_events, half=1, minute_period=(0,3))
# generate the narrative
narrative = wyscout.generate_narrative(filtered_events)
narrative

57 EVENTS IN THIS PERIROD


['(1H) 00:01, G. Wijnaldum of Liverpool made simple pass (accurate) to J. Matip ',
 '(1H) 00:03, J. Matip of Liverpool made simple pass (accurate) to T. Alexander-Arnold ',
 '(1H) 00:07, T. Alexander-Arnold of Liverpool made simple pass (accurate) to Mohamed Salah ',
 '(1H) 00:08, Mohamed Salah of Liverpool made simple pass (accurate) to G. Wijnaldum ',
 '(1H) 00:10, G. Wijnaldum of Liverpool made simple pass (accurate) to T. Alexander-Arnold ',
 '(1H) 00:11, T. Alexander-Arnold of Liverpool made simple pass (accurate) to G. Wijnaldum ',
 '(1H) 00:12, G. Wijnaldum of Liverpool made simple pass (not accurate)  ',
 '(1H) 00:13, Fernandinho of Manchester City made simple pass (interception, accurate) to B. Mendy ',
 '(1H) 00:14, B. Mendy of Manchester City made simple pass (accurate) to Ederson ',
 '(1H) 00:18, Ederson of Manchester City made simple pass (accurate) to Fernandinho ',
 '(1H) 00:19, Fernandinho of Manchester City made simple pass (accurate) to N. Otamendi ',
 '(1H) 00:22, N.

In [21]:
# Aguero scored the first goal, before that, Liverpool had a wave of attacks
filtered_events = wyscout.filter_events_by_time(match_events, half=1, minute_period=(21,24))
narrative = wyscout.generate_narrative(filtered_events)
narrative

57 EVENTS IN THIS PERIROD


['(1H) 21:00, E. Can of Liverpool made simple pass (accurate) to T. Alexander-Arnold ',
 '(1H) 21:03, T. Alexander-Arnold of Liverpool made simple pass (accurate) to Mohamed Salah ',
 '(1H) 21:05, Mohamed Salah of Liverpool made ground attacking duel (take on right, won, accurate)  ',
 '(1H) 21:06, N. Otamendi of Manchester City made ground defending duel (take on left, lost, not accurate)  ',
 '(1H) 21:10, Mohamed Salah of Liverpool made cross (left foot, not accurate)  ',
 '(1H) 21:13, Ederson of Manchester City made touch   ',
 '(1H) 21:28, J. Stones of Manchester City made simple pass (accurate) to Danilo ',
 '(1H) 21:33, Danilo of Manchester City made simple pass (accurate) to Fernandinho ',
 '(1H) 21:34, Fernandinho of Manchester City made simple pass (accurate) to Danilo ',
 '(1H) 21:35, Danilo of Manchester City made simple pass (accurate) to J. Stones ',
 '(1H) 21:39, J. Stones of Manchester City made simple pass (accurate) to Danilo ',
 '(1H) 21:42, Danilo of Manchester City 

You can check the youtube video for the same period here https://youtu.be/5g3bJqWRdIw?t=77.

The narrative generated here still can be improved. For example,
- tags can fits better into the sentence
- different verbs for actions ('made Offside' does not sound good)